In this notebook, different dataset types are explored.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
n_depression = len(dt['user_id'].unique())
print(n_depression)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])

#filtered users
fu = pd.read_csv('filtered_user_lists.csv')
fu_tbt = pd.read_csv('filtered_user_lists_only_tpd_tbt.csv')
fu_maxtime = pd.read_csv('filtered_user_lists_only_tpd_maxtime.csv')
#tpd pairs
tpd_pairs = pd.read_csv('pairs_tpd_users.csv')
filtered_users = pd.concat([fu, fu_tbt, fu_maxtime], ignore_index=True)

603


In [3]:
#naive dataset construction
j = 948
current = filtered_users.iloc[j]
print(current)
current_user_list = eval(current.user_list)

#get distribution of current filtered list
avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day']==avg].count()/len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

#downsample current user list based on distribution
current_samples = pd.DataFrame(columns=['user id', 'tweets per day'])
for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
    frac = float(dist_df.loc[dist_df['tweets per day']==tpd]['percentage'].iloc[0])
    if int(n_depression*frac)<len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression*frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False)

needed_tpd = []
counter = n_depression-len(current_samples['user id'].unique())
if counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            counter = counter - 1
            tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

#upsample/downsample depression cohort accordingly
data = {}
labels = []
for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tweet_list = tmp['created_at'].map(lambda datetime: int(round(datetime.timestamp()))).to_numpy()
    data.update({''+user+'': tweet_list})
    labels.append(1)

for user in current_samples['user id'].unique():
    tmp = st[st['user_id'] == user]
    tweet_list = tmp['created_at'].map(lambda datetime: int(round(datetime.timestamp()))).to_numpy()
    data.update({''+user+'': tweet_list})
    labels.append(0)

df_data = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in data.items()]))
df_labels = pd.DataFrame(labels)
print(df_data)

Unnamed: 0                                                       767
min_tpd                                                           10
max_days_bt                                                        1
min_days_active                                                  360
user_list          ['uR0036', 'uR0129', 'uR0134', 'uR0951', 'uR10...
Name: 767, dtype: object
            uD0000        uD0004        uD0006        uD0007        uD0009  \
0     1.536360e+09  1.536698e+09  1.536673e+09  1.536622e+09  1.536704e+09   
1     1.536355e+09  1.536698e+09  1.536663e+09  1.536521e+09  1.536704e+09   
2     1.536353e+09  1.536697e+09  1.536662e+09  1.536482e+09  1.536704e+09   
3     1.536192e+09  1.536697e+09  1.536661e+09  1.536480e+09  1.536692e+09   
4     1.536192e+09  1.536697e+09  1.536572e+09  1.536479e+09  1.536677e+09   
...            ...           ...           ...           ...           ...   
3221           NaN           NaN           NaN           NaN           NaN   
3222  

In [3]:
# day matrix representation



j = 948
current = filtered_users.iloc[j]
print(current)
current_user_list = eval(current.user_list)

#get distribution of current filtered list
avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day']==avg].count()/len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

#downsample current user list based on distribution
current_samples = pd.DataFrame(columns=['user id', 'tweets per day'])
for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
    frac = float(dist_df.loc[dist_df['tweets per day']==tpd]['percentage'].iloc[0])
    if int(n_depression*frac)<len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression*frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False)

needed_tpd = []
counter = n_depression-len(current_samples['user id'].unique())
if counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            counter = counter - 1
            tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)




Unnamed: 0                                                       767
min_tpd                                                           10
max_days_bt                                                        1
min_days_active                                                  360
user_list          ['uR0036', 'uR0129', 'uR0134', 'uR0951', 'uR10...
Name: 767, dtype: object


In [9]:
# list of matrices with different lenght of observed days | row = time-point, col = variable (ith h of day)
data = {}
labels = []
data_list = []

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]

    timestamps = tmp['created_at'].iloc[::-1] #sort dates in ascending order
    n_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique()) #number of recorded days
    matrix = np.full((n_days,24), 0) #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0,n_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date())==timestamps.iloc[d].date()] #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour # user hour of timestamp as index in matrix
            matrix[d,hour_i]=matrix[d,hour_i]+1 # move through matrix and add tweets per hour

        last_date = current_date

    data.update({''+user+'': matrix})
    data_list.append(matrix)
    labels.append(1)

data

{'uD0000': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0004': array([[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0006': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0007': array([[0, 2, 4, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0009': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],

In [14]:
# list of matrices with longest lenght of observed days for all -> very sparse | row = time-point, col = variable (ith h of day)
data = {}
labels = []
data_list = []

n_days = 0
for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique()) #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique()) #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1] #sort dates in ascending order
    matrix = np.full((n_days,24), 0) #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0,tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date())==timestamps.iloc[d].date()] #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour # user hour of timestamp as index in matrix
            matrix[d,hour_i]=matrix[d,hour_i]+1 # move through matrix and add tweets per hour

        last_date = current_date

    data.update({''+user+'': matrix})
    data_list.append(matrix)
    labels.append(1)

data

{'uD0000': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0004': array([[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0006': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0007': array([[0, 2, 4, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'uD0009': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],

In [16]:
# list of dfs with different lenght of observed days | row = time-point, col = variable (ith h of day)
data = {}
labels = []
data_list = []

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]

    timestamps = tmp['created_at'].iloc[::-1] #sort dates in ascending order
    n_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique()) #number of recorded days
    matrix = np.full((n_days,24), 0) #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0,n_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date())==timestamps.iloc[d].date()] #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour # user hour of timestamp as index in matrix
            matrix[d,hour_i]=matrix[d,hour_i]+1 # move through matrix and add tweets per hour

        last_date = current_date

    tmp_df = pd.DataFrame(matrix)
    data_list.append(tmp_df)
    labels.append(1)

data_list

[     0   1   2   3   4   5   6   7   8   9   ...  14  15  16  17  18  19  20  \
 0     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   1   
 1     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 2     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 3     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 4     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 ..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
 232   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 233   0   0   0   0   1   0   0   0   0   0  ...   0   0   0   0   1   1   1   
 234   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 235   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 236   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 
      21  22  23  
 0     